# Interface between Graphene and SiO2 (

## Introduction

This notebook demonstrates the creation of an interface between 2D and 3D material: Graphene on SiO2. 

Following the manuscript:
    > **Yong-Ju Kang, Joongoo Kang, and K. J. Chang** 
    > "Electronic structure of graphene and doping effect on SiO2"
    > Phys. Rev. B 78, 115404 (2008)
    > [DOI: 10.1103/PhysRevB.78.115404](https://doi.org/10.1103/PhysRevB.78.115404)



Replicating the materials from the manuscript, FIG. 1. (b):

<img src='https://i.imgur.com/ltSj5uk.png' width='600'/>


## 1. Prepare the Environment
### 1.1. Set up the notebook 

Set the following flags to control the notebook behavior 

In [ ]:
FILM_INDEX = 1  # Index in the list of materials, to access as materials[FILM_INDEX]
FILM_MILLER_INDICES = (0, 0, 1)
FILM_THICKNESS = 1  # in atomic layers
FILM_VACUUM = 0.0  # in angstroms
FILM_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
FILM_USE_ORTHOGONAL_Z = True

SUBSTRATE_INDEX = 0
SUBSTRATE_MILLER_INDICES = (0, 0, 1)
SUBSTRATE_THICKNESS = 7  # in atomic layers (for 14 bilayers -- from manuscript)
SUBSTRATE_VACUUM = 0.0  # in angstroms
SUBSTRATE_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
SUBSTRATE_USE_ORTHOGONAL_Z = True

# Maximum area for the superlattice search algorithm
MAX_AREA = 150  # in Angstrom^2
# Set the termination pair indices
TERMINATION_PAIR_INDICES = [1]  # For O-terminated
INTERFACE_DISTANCE = 2.58  # in Angstrom -- from manuscript
INTERFACE_VACUUM = 20.0  # in Angstrom -- from manuscript

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("specific_examples|create_interface_with_min_strain_zsl.ipynb")

### 1.3. Get input materials and assign `substrate` and `film`

In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

film = Material(Materials.get_by_name_first_match("Graphene"))
substrate = Material(Materials.get_by_name_first_match("SiO2"))

### 1.4. Preview Substrate and Film

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([substrate, film], repetitions=[3, 3, 1], rotation="0x")

## 2. Configure slabs and select termination pair

### 2.1. Create Substrate and Layer Slabs
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.
Define the substrate slab cell that will be used as a base for the interface and the film slab cell that will be placed on top of the substrate slab.

In [ ]:
from mat3ra.made.tools.build.slab import SlabConfiguration, get_terminations, create_slab, \
    PymatgenSlabGeneratorParameters

film_slab_configuration = SlabConfiguration(
    bulk=film,
    miller_indices=FILM_MILLER_INDICES,
    thickness=FILM_THICKNESS,  # in atomic layers
    vacuum=FILM_VACUUM,  # in angstroms
    xy_supercell_matrix=FILM_XY_SUPERCELL_MATRIX,
    use_orthogonal_z=FILM_USE_ORTHOGONAL_Z
)

substrate_slab_configuration = SlabConfiguration(
    bulk=substrate,
    miller_indices=SUBSTRATE_MILLER_INDICES,
    thickness=SUBSTRATE_THICKNESS,  # in atomic layers
    vacuum=SUBSTRATE_VACUUM,  # in angstroms
    xy_supercell_matrix=SUBSTRATE_XY_SUPERCELL_MATRIX,
    use_orthogonal_z=SUBSTRATE_USE_ORTHOGONAL_Z,
)

params = PymatgenSlabGeneratorParameters(
    symmetrize=False)

### 2.2. Get possible terminations for the slabs

In [ ]:
film_slab_terminations = get_terminations(film_slab_configuration, params)
substrate_slab_terminations = get_terminations(substrate_slab_configuration, params)

### 2.3. Visualize slabs for all possible terminations

In [ ]:
film_slabs = [create_slab(film_slab_configuration, termination) for termination in film_slab_terminations]
substrate_slabs = [create_slab(substrate_slab_configuration, termination, params) for termination in
                   substrate_slab_terminations]

visualize([{"material": slab, "title": slab.metadata["build"]["termination"]} for slab in film_slabs],
          repetitions=[3, 3, 1], rotation="-90x")
visualize([{"material": slab, "title": slab.metadata["build"]["termination"]} for slab in substrate_slabs],
          repetitions=[3, 3, 1], rotation="-90x")

### 2.4. Print terminations for the interface

In [ ]:
from itertools import product

termination_pairs = list(product(film_slab_terminations, substrate_slab_terminations))
print("Termination Pairs (Film, Substrate)")
for idx, termination_pair in enumerate(termination_pairs):
    print(f"    {idx}: {termination_pair}")

## 3. Create interfaces

### 3.1. Initialize the Interface Configuration

In [ ]:
from mat3ra.made.tools.build.interface import InterfaceConfiguration
from mat3ra.made.tools.build.interface import ZSLStrainMatchingParameters
from mat3ra.made.tools.build.interface import ZSLStrainMatchingInterfaceBuilder, \
    ZSLStrainMatchingInterfaceBuilderParameters

interfaces = []
for termination_pair_idx in TERMINATION_PAIR_INDICES:
    termination_pair = termination_pairs[termination_pair_idx]
    film_termination, substrate_termination = termination_pair
    interface_configuration = InterfaceConfiguration(
        film_configuration=film_slab_configuration,
        substrate_configuration=substrate_slab_configuration,
        film_termination=film_termination,
        substrate_termination=substrate_termination,
        distance_z=INTERFACE_DISTANCE,
        vacuum=INTERFACE_VACUUM
    )

    zsl_strain_matching_parameters = ZSLStrainMatchingParameters(
        max_area=MAX_AREA,
    )

    matched_interfaces_builder = ZSLStrainMatchingInterfaceBuilder(
        build_parameters=ZSLStrainMatchingInterfaceBuilderParameters(
            strain_matching_parameters=zsl_strain_matching_parameters))
    interfaces_sorted_by_size_and_strain = matched_interfaces_builder.get_materials(
        configuration=interface_configuration)
    selected_interface = interfaces_sorted_by_size_and_strain[0]
    selected_interface.name = f"Interface {termination_pair}"
    interfaces.append(selected_interface)


## 4. Visualize the interfaces

In [ ]:
visualize(interfaces, repetitions=[3, 3, 1])
visualize(interfaces, repetitions=[1, 1, 1], rotation="-90x")

## 5. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import download_content_to_file

for idx, interface in enumerate(interfaces):
    download_content_to_file(interfaces, f"interface_{idx}.json")